#Init and load data

In [ ]:
#In this cell, there are various option for the user to freely choose:
#embedding_choice
#PREPROCESSING_CHOICE

import pandas as pd
import numpy as np

use_drive = True
#for euler, remove this entire if else branch and set PATH to "./"
if use_drive:
  PATH = "/content/drive/MyDrive/CIL 2022/"
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/CIL 2022/
  !ls
else:
  PATH = "./"


print("Choosing data: ", end="")
#option 0 - 8
PREPROCESSING_OPTIONS = [ "raw",
"no-stemming_no-lemmatize_no-stopwords_no-spellcorrect",
"no-stemming_no-lemmatize_with-stopwords_no-spellcorrect",
"no-stemming_no-lemmatize_with-stopwords_with-spellcorrect",
"no-stemming_with-lemmatize_with-stopwords_no-spellcorrect",
"no-stemming_with-lemmatize_with-stopwords_with-spellcorrect",
"with-stemming_no-lemmatize_with-stopwords_no-spellcorrect",
"with-stemming_with-lemmatize_no-stopwords_with-spellcorrect",
"with-stemming_with-lemmatize_with-stopwords_no-spellcorrect" ]
PREPROCESSING_CHOICE = PREPROCESSING_OPTIONS[8] # one from PREPROCESSING_OPTIONS
print(PREPROCESSING_CHOICE)



print("Init. methods.")
#helper function to save models and stats
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import pickle

#save model
# now you can save it to a file
# with open('filename.pkl', 'wb') as f:
#     pickle.dump(clf, f)

# # and later you can load it
# with open('filename.pkl', 'rb') as f:
#     clf = pickle.load(f)

#Saving trained model
def save_model(model, model_name):
  name_of_file_model = "trained " + model_name + ".pkl"
  with open(name_of_file_model, 'wb') as f:
    pickle.dump(model, f)
  print("Model saved.")

#Saving stats of model to a file
def save_stats_to_file(y_val, y_val_pred, model_name):
  name_of_file_stats = model_name + PREPROCESSING_CHOICE + ".txt"
  # name_of_file_model = "trained " + model_name + ".pkl"
  # print("Saving model to file.")
  # with open(name_of_file_model, 'wb') as f:
  #   pickle.dump(model, f)
  print("Saving model relevant stats to file.")
  file_obj = open(name_of_file_stats, "w", encoding="utf8")
  file_obj.write(f'use_drive: {use_drive}\n')
  # file_obj.write(f'LOAD_FROM_SPLITTED_DATASET: {LOAD_FROM_SPLITTED_DATASET}\n')
  # file_obj.write(f'WRITE_SPLITTED_DATASET: {WRITE_SPLITTED_DATASET}\n')
  # file_obj.write(f'REMOVE_DUPLICATES_FROM_TRAINING: {REMOVE_DUPLICATES_FROM_TRAINING}\n')
  file_obj.write(f'PREPROCESSING_CHOICE: {PREPROCESSING_CHOICE}\n')
  file_obj.write(f'Model name: {model_name}\n')
  file_obj.write(f'Acc: {accuracy_score(y_val, y_val_pred)}\n')
  file_obj.write(f'Recall: {recall_score(y_val, y_val_pred)}\n')
  file_obj.write(f'Precision: {precision_score(y_val, y_val_pred)}\n')
  file_obj.write(f'F1: {f1_score(y_val, y_val_pred)}\n')
  file_obj.write(f'ROC_AUC: {roc_auc_score(y_val, y_val_pred)}\n')
  file_obj.close()
  print(f'Acc: {accuracy_score(y_val, y_val_pred)}')
  print(f'Recall: {recall_score(y_val, y_val_pred)}')
  print(f'Precision: {precision_score(y_val, y_val_pred)}')
  print(f'F1: {f1_score(y_val, y_val_pred)}')
  print(f'ROC_AUC: {roc_auc_score(y_val, y_val_pred)}')

def read_file_and_strip(filename):
  lines = []
  with open(filename) as file:
    for line in file:
      lines.append(line.strip())
  return np.array(lines)

def read_data():
  dataset_path = PATH + "data/" + PREPROCESSING_CHOICE + "/"

  train_sentences = read_file_and_strip(dataset_path + "train_sentences.txt")
  train_labels = read_file_and_strip(dataset_path + "train_labels.txt").astype(int)
  val_sentences = read_file_and_strip(dataset_path + "val_sentences.txt")
  val_labels = read_file_and_strip(dataset_path + "val_labels.txt").astype(int)
  
  return train_sentences, train_labels, val_sentences, val_labels

print("Loading data.")

train_sentences, train_labels, val_sentences, val_labels = read_data()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1NSBJKoA0FA5BeDWR8bNo5tPgA5lCU8yv/CIL 2022
'Bert inspiration'
 BoW_v2.ipynb
 code
 data
 Models
 Notes.gdoc
 On_Processed_Bag-of-words_baseline.ipynb
 papers_to_possible_reference
 Preprocessing
 Project_2_Bag-of-words_baseline.ipynb
 __pycache__
 resources
 Results.gsheet
 template.ipynb
'trained Glove_LogisticRegression(C=1, max_iter=1000).pkl'
 trash
'Word embeddings'
 XGBoost
Choosing data: with-stemming_with-lemmatize_with-stopwords_no-spellcorrect
Init. methods.
Loading data.


#XGBoost Classifier

##XGBoost Classifier + CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

features = 100000
vectorizer = CountVectorizer(max_features = features)
#vectorizer = TfidfVectorizer()


X_train = vectorizer.fit_transform(train_sentences)
X_val = vectorizer.transform(val_sentences)

y_train = train_labels
y_val = val_labels

# print(len(X_train))
# print(len(y_train))
# print(vectorizer.shape)

In [ ]:
# XGB Classifier on training data

from xgboost import XGBClassifier
import time

depth = 12
estimators = 1024 + 512
model_name = "Features=" + str(features) + " " + "XGB Classifier" + " estimators=" + str(estimators) + " depth=" + str(depth)
#model_name = str(features) + " " + "XGB Classifier" + " " + str(depth) + " " + str(estimators)
#model_name = "Features=" + str(features) + " " + "XGB Classifier" + " estimators=" + str(estimators)
#model_name = "TfidfVectorizer -" + "XGB Classifier" + " estimators=" + str(estimators)
#model_name = "TfidfVectorizer -" + "XGB Classifier" + " estimators=" + str(estimators) + " depth=" + str(depth)

start_time = time.time()
#xgb_model = XGBClassifier(max_depth=depth, n_estimators=estimators, n_jobs=2, random_state=42).fit(X_train, y_train) 
xgb_model = XGBClassifier(n_estimators=estimators, max_depth=depth, n_jobs=2, random_state=42).fit(X_train, y_train) 
predicted_y_label = xgb_model.predict(X_val)
end_time = time.time()
elapsed_time_in_seconds = end_time - start_time
print("Training and evaluation duration: %d s" %elapsed_time_in_seconds)

model_name = str(elapsed_time_in_seconds) + "s - " + model_name
save_stats_to_file(y_val, predicted_y_label, model_name)
save_model(xgb_model, model_name)

Training and evaluation duration: 9669 s
Saving model relevant stats to file.
Acc: 0.8078775818294106
Recall: 0.8500054812541109
Precision: 0.7843757112870186
F1: 0.8158728906075313
ROC_AUC: 0.8078133077436401
Model saved.


##XGBoost Classifier + TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# features = 100000
# vectorizer = CountVectorizer(max_features = features)
vectorizer = TfidfVectorizer()


X_train = vectorizer.fit_transform(train_sentences)
X_val = vectorizer.transform(val_sentences)

y_train = train_labels
y_val = val_labels

# print(len(X_train))
# print(len(y_train))
# print(vectorizer.shape)

In [ ]:
# XGB Classifier on training data

from xgboost import XGBClassifier
import time

depth = 12
estimators = 1024 + 512
model_name = "Features=" + str(features) + " " + "XGB Classifier" + " estimators=" + str(estimators) + " depth=" + str(depth)
#model_name = str(features) + " " + "XGB Classifier" + " " + str(depth) + " " + str(estimators)
#model_name = "Features=" + str(features) + " " + "XGB Classifier" + " estimators=" + str(estimators)
#model_name = "TfidfVectorizer -" + "XGB Classifier" + " estimators=" + str(estimators)
#model_name = "TfidfVectorizer -" + "XGB Classifier" + " estimators=" + str(estimators) + " depth=" + str(depth)

start_time = time.time()
#xgb_model = XGBClassifier(max_depth=depth, n_estimators=estimators, n_jobs=2, random_state=42).fit(X_train, y_train) 
xgb_model = XGBClassifier(n_estimators=estimators, max_depth=depth, n_jobs=2, random_state=42).fit(X_train, y_train) 
predicted_y_label = xgb_model.predict(X_val)
end_time = time.time()
elapsed_time_in_seconds = end_time - start_time
print("Training and evaluation duration: %d s" %elapsed_time_in_seconds)

model_name = str(elapsed_time_in_seconds) + "s - " + model_name
save_stats_to_file(y_val, predicted_y_label, model_name)
save_model(xgb_model, model_name)

Training and evaluation duration: 6574 s
Saving model relevant stats to file.
Acc: 0.8047073354834282
Recall: 0.8486260323028576
Precision: 0.7805300216781219
F1: 0.8131548770559966
ROC_AUC: 0.804640329197218
Model saved.
